### Homework

* 1 - Bajar el CSV de Oferta Gastronómica desde Buenos Aires Data (https://data.buenosaires.gob.ar/dataset/). Idealmente hacer esto con Python.
* 2 - Crear una tabla con los siguientes campos: id_local, nombre, categoria, direccion, barrio, comuna, para posteriormente poblarla con los datos bajados, utilzando el conector desde el script de Python.
* 3 - A partir de tener los datos disponibles, responder a las siguientes preguntas:

  * a) ¿Cuál es el barrio con mayor cantidad de Pubs?
  * b) Obtener la cantidad de locales por categoría
  * c) Obtener la cantidad de restaurantes por comuna

In [1]:
import pandas as pd
import numpy as np
import pymysql, requests, io
from Pass_SQL import contraseña

### 1) Request Dataframe locales gastronomicos BA
Se usa la libreria requests y los metodos .get(URL) y .text  


In [3]:
respuesta = requests.get('https://cdn.buenosaires.gob.ar/datosabiertos/datasets/ente-de-turismo/oferta-establecimientos-gastronomicos/oferta_gastronomica.csv')
df_locales = pd.read_csv(io.StringIO(respuesta.text))

In [5]:
#Reemplazo valores nan por None
df_locales.replace(np.nan, None, inplace=True)

In [6]:
campos = ['nombre','categoria','direccion_completa','barrio','comuna']
lista_locales = [tuple(df_locales[campos].loc[fila]) for fila in range(0,len(df_locales))]

### 2) Conexión a MySQL e ingreso de datos
Se usa la libreria pymysql, se crea la conexión y el cursos. Luego se ejecutan las queries y commit

In [7]:
#CONEXION
conexion = pymysql.connect(
    host = 'localhost',
    user = 'root',
    password = contraseña(),
    port = 3306,
    database = 'Oferta_gastronomica'
)
#CURSOR
cursor = conexion.cursor()

In [ ]:
#INSERCIÓN DE FILAS EN TABLA locales y COMMIT
cursor.executemany('''INSERT INTO locales (nombre,categoria,direccion,barrio,comuna) 
                      VALUES (%s,%s,%s,%s,%s)''',lista_locales)
conexion.commit()

* 3 - A partir de tener los datos disponibles, responder a las siguientes preguntas:

  * a) ¿Cuál es el barrio con mayor cantidad de Pubs?
  * b) Obtener la cantidad de locales por categoría
  * c) Obtener la cantidad de restaurantes por comuna

In [ ]:
# a) Barrio con mayor cantidad de pubs -->
cursor.execute("""SELECT barrio, categoria, count(*) as Cantidad
                  FROM locales
                  GROUP BY barrio
                  HAVING categoria = 'PUB'
                  ORDER BY Cantidad DESC;""")
cursor.fetchall()

In [ ]:
# b) Cantidad de locales por categoría --> 
cursor.execute("""SELECT categoria, count(*) as Cantidad_locales
                  FROM locales
                  GROUP BY categoria;""")
cursor.fetchall()

In [ ]:
# c) Obtener la cantidad de restaurantes por comuna -->
cursor.execute('''SELECT comuna, count(*) as Cantidad_restaurantes
                  FROM locales
                  WHERE categoria like '%RESTAURANTE%'
                  GROUP BY comuna
                  ORDER BY Cantidad_restaurantes DESC;''')
cursor.fetchall()

In [ ]:
conexion.close()